# IMPORTS

In [1]:
%matplotlib inline

In [2]:
from AxonDeepSeg.visualization.visualize import visualize_training
import json
import os
import matplotlib.pyplot as plt
#from prettytable import PrettyTable
#from ipywidgets.widgets import *

# 1/ Build the dataset for training.

In [3]:
# input data to build the training set
dataset_name = 'SEM_2classes_reduced'

In [4]:
path_data = '../data/' + dataset_name + '/raw/'
path_testing = '../data/' + dataset_name + '/testing/pilou/'

# output path of training data path
path_training = '../data/' + dataset_name + '/training/'

# 2/ Load the config file and train the network

Choose here the name of the model you want to load

In [5]:
model_name = 'baseline'

In [6]:
# optional input path of a model to initialize the training
#path_model_init = 'network_testing/test_2905'
path_model_init = None

# output path for trained U-Net
path_model = '../models/' + model_name

path_configfile = path_model + '/config_network.json'

if not os.path.exists(path_model):
    os.makedirs(path_model)

with open(path_configfile, 'r') as fd:
    config_network = json.loads(fd.read())
    
# OPTIONAL : specify the gpu one wants to use.
gpu_device = 'gpu:0' # or gpu_device = 'gpu:1' these are the only two possible targets for now.

# TRAINING OF THE NETWORK

Uniquement nécessaire si le réseau n'a pas encore été entrainé. Sinon on va juste utiliser le modèle existant dans models

from AxonDeepSeg.train_network import train_model
train_model(path_training, path_model, config_network, path_model_init=None, gpu = gpu_device)

# 3/ Apply the model to segment one image

#### Segmentation

In [7]:
from AxonDeepSeg.apply_model import axon_segmentation
axon_segmentation(path_testing, path_model, config_network)



 ---Start axon segmentation on ../data/SEM_2classes_reduced/testing/pilou/---
hyperparameters detected in the model


/home/GRAMES.POLYMTL.CA/mawab/ads/tf11-ads/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


KeyError: 'network_batch_norm_decay'

# VISUALISATION OF THE SEGMENTATION

Imports

In [ ]:
imorg = plt.imread(path_testing + '/image.png')
imads = plt.imread(path_testing + '/AxonDeepSeg.png')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Original image')
plt.imshow(imorg, cmap='gray')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Segmented image')
plt.imshow(imads, cmap='gray')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Superposed images')
plt.imshow(imorg, cmap='gray', alpha=0.5)
plt.imshow(imads, cmap='viridis', alpha=0.5)